In [1]:
from slackclient import SlackClient
import psycopg2 as pg
import pandas.io.sql as psql
#from datetime import datetime
import datetime
import sys
import json
import copy
import io

print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)
conn_str = "host={0} port={1} dbname={2} user={3} password={4}" \
.format("localhost", 5432, "kanjidb", "postgres", "w0lfpack")

try:
    conn = pg.connect(conn_str)
    print("Welcome to Jupyter Notebook.  You are connected to the Kanji  database!")
except pg.OperationalError:
    print("You are not connected to the database.")

Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji  database!


# Post Piglet Squeal alerts to Slack #
* each location has a different Slack channel *


In [5]:
_BANNER_ALERT_MESSAGE = "Node Alert"
_BANNER_OK_MESSAGE = "Sensor Network"
_SLACK_EMOJI = ':warning:'

_STATUS_PIGLET_SQUEAL = 100
_REPORT_INTERVAL_MINS = 100
_DEPLOY_STATE_MAINTENANCE = 10000

In [6]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
	} ]"

nodetemplate = "{\"type\": \"context\", \
		\"elements\": [ \
			{ \
				\"type\": \"image\", \
				\"image_url\": \"https://www.dropbox.com/s/o2gjgfsph2cxgds/thermometer.jpg?raw=1\", \
				\"alt_text\": \"notifications warning icon\" \
			}, \
			{ \
				\"type\": \"mrkdwn\", \
				\"text\": \"*Conflicts with Team Huddle: 4:15-4:30pm*\" \
			} \
		] \
	}"

dividertemplate = "{ \
		\"type\": \"divider\" \
	} "


#locationimageurl = 'https://www.dropbox.com/s/k8td4cmja7vs30r/building.jpg?raw=1'

In [8]:
nodemessage  = json.loads(nodetemplate)
divider = json.loads(dividertemplate)
now = datetime.datetime.now()

_SQUEALTAPE_ROOT_DIRECTORY = '/home/sensei/senseiweb/kanji-static/media/squealtape_'

locationsquery = "SELECT location.idlocation, location.description, location.imageurl, customer.slacktoken FROM kanji_location location \
JOIN kanji_customer customer ON location.customer_id=customer.idcustomer;"

#print(locationsquery)

cur = conn.cursor()
cur.execute(locationsquery)
locationrows = cur.fetchall()

for locationrow in locationrows: 
  blockmessage = json.loads(messagetemplate)
  #print(locationrow)
  locationid = locationrow[0]
  location = locationrow[1]
  locationimageurl = locationrow[2]
  _SLACK_TOKEN = locationrow[3]
  slackchannel = "sounds"

  locationimageurl = "https://www.dropbox.com/s/rxkbzreygu92rak/sound-icon.jpg?raw=1" 

  #query = "SELECT pinglog.node_id, node.name, node.friendlyname, node.deploystate_id, COUNT(*) AS count FROM kanji_pinglog pinglog \
  #JOIN kanji_node node ON pinglog.node_id=idnode \
  #WHERE pinglog.pingstate_id={0} AND pinglog.timestamp>= NOW() - INTERVAL '{1} minutes' \
  #AND node.location_id={2} \
  #GROUP BY (pinglog.node_id, node.name, node.friendlyname, node.deploystate_id) ORDER BY COUNT(*);".format(_STATUS_PIGLET_SQUEAL, _REPORT_INTERVAL_MINS, locationid)

  query = "SELECT eventlog.node_id, node.name, node.deploystate_id, eventlog.eventdata FROM kanji_eventlog eventlog \
  JOIN kanji_node node ON eventlog.node_id=idnode \
  WHERE eventlog.sensortype_id={} AND eventlog.timestamp>= NOW() - INTERVAL '{} minutes' \
  AND node.location_id={}".format(_STATUS_PIGLET_SQUEAL, _REPORT_INTERVAL_MINS, locationid)

  print(query)

  cur = conn.cursor()
  cur.execute(query)
  rows = cur.fetchall()

  totalfailures = 0

  print("\nPing Report for location {}".format(location))
  if len(rows) > 0:
    #print("\nPing Report for location {}".format(location))
    reportnodes = []    
    print("The query returned the following:") 
    for row in rows:
        #print(row)
        node = {}    
        node['id'] = row[0]
        node["name"] = row[1]
        node['friendlyname'] = "Piggy"
        if node['friendlyname'] is None:
          node['friendlyname'] = row[1]
        node['deploystate'] = row[2]
        eventdata = json.loads(row[3])
        squealtapename = eventdata[0]['f']
        squealhits = int(eventdata[0]['h'])
        print("eventdata = {}".format(eventdata))
        
        node['deploystate'] = 0
        node['level'] = ""
                
        reportnodes.append(node)    

        # start key replacement
        # the troubling image
        #print(blockmessage[0]["accessory"]["image_url"])
        blockmessage[0]["accessory"]["image_url"] = locationimageurl

        # *Porter Farms - Building 1  / 02:15 PM Monday, March 18, 2019 
        # The current building temperature is 69.2° F
        blockmessage[0]["text"]["text"] = "*{}* \
          \nAt {} \n*Piglet Squeal*".format(location, now.strftime("%-I:%M %p %A, %B %e, %Y"))

        #or reportnode in reportnodes:
        #print("reportnodename={0}".format(reportnode["name"]))
        thisnodemessage = copy.deepcopy(nodemessage)
        alerticonurl = "https://www.dropbox.com/s/khe23ou4myd0w5q/elevated.png?raw=1"        
        thisnodemessage["elements"][0]["image_url"] = alerticonurl
        thisnodemessage["elements"][1]["text"] = "{0} / {1}".format(node['friendlyname'], node["level"])
        blockmessage.append(thisnodemessage.copy())  
        blockmessage.append(divider)
    
        sc = SlackClient(_SLACK_TOKEN)
        response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
        print(blockmessage) 
        if not 'ok' in response or not response['ok']:
          print("Error posting message to Slack channel")
          print(blockmessage)
          print(response)
        else:
          print("Ok posting message to Slack channel")
          print(_SQUEALTAPE_ROOT_DIRECTORY + squealtapename)
          with open(_SQUEALTAPE_ROOT_DIRECTORY + squealtapename, 'rb') as f:
            response = sc.api_call("files.upload", channels=slackchannel, thread_ts=response['ts'], filename=squealtapename, file=io.BytesIO(f.read()))
          if not 'ok' in response or not response['ok']:
            print("Error posting squealtape to Slack channel")
            print(response)
          else:
            print("Ok posting squealtape")
            print(response)
  else:
    print("There are no piglet alerts in this period.")
    
print("\nDone!")

SELECT eventlog.node_id, node.name, node.deploystate_id, eventlog.eventdata FROM kanji_eventlog eventlog   JOIN kanji_node node ON eventlog.node_id=idnode   WHERE eventlog.sensortype_id=100 AND eventlog.timestamp>= NOW() - INTERVAL '100 minutes'   AND node.location_id=10002

Ping Report for location Pig Farm
There are no piglet alerts in this period.
SELECT eventlog.node_id, node.name, node.deploystate_id, eventlog.eventdata FROM kanji_eventlog eventlog   JOIN kanji_node node ON eventlog.node_id=idnode   WHERE eventlog.sensortype_id=100 AND eventlog.timestamp>= NOW() - INTERVAL '100 minutes'   AND node.location_id=10019

Ping Report for location Pig Farm (outdoors)
There are no piglet alerts in this period.
SELECT eventlog.node_id, node.name, node.deploystate_id, eventlog.eventdata FROM kanji_eventlog eventlog   JOIN kanji_node node ON eventlog.node_id=idnode   WHERE eventlog.sensortype_id=100 AND eventlog.timestamp>= NOW() - INTERVAL '100 minutes'   AND node.location_id=10022

Ping Re